In [3]:
import math
import numpy as np
import joblib
from pathlib import Path
import pickle
# MACHINE LEARNING LIBRARIES
import sklearn
from sklearn.metrics import classification_report,balanced_accuracy_score
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
try:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
  # Invalid device or cannot modify virtual devices once initialized.
  pass
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import TimeDistributed
import tensorflow_addons as tfa
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
# OPTUNA
import optuna
from optuna.trial import TrialState
# CUSTOM LIBRARIES

from video_gen import FrameGenerator

from resnet183d import resnet3d
from transformer import TransformerEncoder, pos_encoding,MultiHeadAttention,TubeletEmbedding,TransformerDecoder,Transformerfusion,Transformerfusion2
from data import  random_flip, random_noise, one_hot,aug,combine_inputs
from tools import CustomSchedule, CosineSchedule
from tools import Logger,read_yaml

import copy
import random
import os
from glob import glob
from os.path import join,exists

tf.random.set_seed(1234)
np.random.seed(1234)
random.seed(1234)


2023-01-26 02:54:04.747066: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-26 02:54:05.296098: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ros/foxy/opt/yaml_cpp_vendor/lib:/opt/ros/foxy/opt/rviz_ogre_vendor/lib:/opt/ros/foxy/lib/x86_64-linux-gnu:/opt/ros/foxy/lib:/usr/local/cuda-11.7/lib64${LD_LIBRARY_PATH:+:}
2023-01-26 02:54:05.296261: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object fi

In [4]:
# TRAINER CLASS 
class Trainer:
    def __init__(self, config, logger):
        self.config = config
        self.logger = logger
        self.split = 1
        self.fold = 0
        self.trial = None
        self.dataset=config['dataset']
        self.features=config['features']
        self.evaluation=config['evalutaion']
        self.num_class=self.config[self.evaluation]['CLASSES']
        if self.features=='skeleton':
            self.pose=config[self.features]['pose']
            
            
            self.dataset_dire_pose=join('../features/',self.dataset,self.features,self.pose,self.evaluation)

            self.shape=(32,34)

        elif self.features=='rgb':
            self.backbone=self.config[self.features]['backbone']
            self.dataset_dire_rgb=join('../features/',self.dataset,self.features,self.backbone,self.evaluation)
            self.shape=(32,350,350,3)

        else:
            self.pose=config[self.features]['pose']
            
            
            
            self.dataset_dire_pose=join('../features/',self.dataset,'skeleton',self.pose,self.evaluation)

            self.shape_pose=(32,34)

            self.backbone=self.config[self.features]['backbone']
            self.dataset_dire_rgb=join('../features/',self.dataset,'rgb',self.backbone,self.evaluation,'crop')
            self.shape_rgb=(32,1024)

            self.arch=config[self.features]['arch']

            

            

            
        self.result_dire=self.config['Result_dire']    




        self.bin_path = self.config['MODEL_DIR']
        
        self.model_size = self.config['MODEL_SIZE']
        self.n_heads = self.config[self.model_size]['N_HEADS']
        #self.n_heads = 256
        self.n_layers = self.config[self.model_size]['N_LAYERS']
        self.embed_dim = self.config[self.model_size]['EMBED_DIM']
        self.dropout = self.config[self.model_size]['DROPOUT']
        self.mlp_head_size = self.config[self.model_size]['MLP']
        self.activation = tf.nn.gelu
        self.d_model = 1 * self.n_heads
        
        self.d_ff = self.d_model * 2 



    def build_act(self,transformer):
        inputs = tf.keras.layers.Input(shape=self.shape)
        

        

        

        # x=tf.keras.layers.LSTM(units=self.d_model,return_sequences=True)(inputs)
        
        # x=tf.keras.layers.LayerNormalization()(x)

        
        

    
        
        x = transformer(inputs,mask=None,training=True)



        
        x = tf.keras.layers.Lambda(lambda x: x[:,0,:])(x)
        #x = tf.keras.layers.Dense(self.mlp_head_size)(x)
        outputs = tf.keras.layers.Dense(self.num_class)(x)
        return tf.keras.models.Model(inputs, outputs)
        

    def build_fusion(self,transformer_fusion):
        input_pose = tf.keras.layers.Input(shape=self.shape_pose)
        input_rgb=  tf.keras.layers.Input(shape=self.shape_rgb)
        

        

        

        # x=tf.keras.layers.LSTM(units=self.d_model,return_sequences=True)(inputs)
        
        # x=tf.keras.layers.LayerNormalization()(x)

        
        

    
        
        # x = transformer_fusion([input_rgb,input_pose],mask=None,training=True)
        x = transformer_fusion([input_pose,input_rgb],mask=None,training=True)
        

       
        
        x = tf.keras.layers.Lambda(lambda x: x[:,0,:])(x)
        #x = tf.keras.layers.Dense(self.mlp_head_size)(x)
        outputs = tf.keras.layers.Dense(self.num_class)(x)
        return tf.keras.models.Model([input_pose,input_rgb], outputs)


    def build_rgb_BERT(self,resnet):
        

        inputs=tf.keras.Input(shape=(32,350,350,3))

        x=resnet(inputs)

        # x = transformer(x,mask=None,training=True)


        
        # x = tf.keras.layers.Lambda(lambda x: x[:,0,:])(x)
        # #x = tf.keras.layers.Dense(self.mlp_head_size)(x)
        outputs = tf.keras.layers.Dense(self.num_class)(x)
        return tf.keras.models.Model(inputs, outputs)



    


    def get_model(self,trial_num):

       
        

        if self.features=='skeleton':
            transformer = TransformerEncoder(d_model=self.d_model,dff=self.d_ff,num_heads=self.n_heads,rate=self.dropout,num_layers=self.n_layers)
            self.model = self.build_act(transformer)
            lr = CustomSchedule(self.d_model, 
                    warmup_steps=len(self.ds_train_pose)*self.config['N_EPOCHS']*self.config['WARMUP_PERC'],
                    decay_step=len(self.ds_train_pose)*self.config['N_EPOCHS']*self.config['STEP_PERC'])
        elif self.features=='rgb':
            #transformer = TransformerEncoder(d_model=self.d_model,dff=self.d_ff,num_heads=self.n_heads,rate=self.dropout,num_layers=self.n_layers)
            if self.backbone=='raw':
                resnet=resnet3d(self.d_model)
                
                self.model=self.build_rgb_BERT(resnet)
            else:
                self.model = self.build_act(transformer)

            lr = CustomSchedule(self.d_model, 
                warmup_steps=80*self.config['N_EPOCHS']*self.config['WARMUP_PERC'],
                decay_step=80*self.config['N_EPOCHS']*self.config['STEP_PERC'])


        else:
            #transformer = TransformerEncoder(d_model=self.d_model,dff=self.d_ff,num_heads=self.n_heads,rate=self.dropout,num_layers=self.n_layers)
            transformer_fusion=Transformerfusion2(d_model=self.d_model,dff=self.d_ff,num_heads=self.n_heads,rate=self.dropout,num_layers=self.n_layers)
            self.model=self.build_fusion(transformer_fusion)


        
            lr = CustomSchedule(self.d_model, 
                warmup_steps=len(self.ds_train_fusion)*self.config['N_EPOCHS']*self.config['WARMUP_PERC'],
                decay_step=len(self.ds_train_fusion)*self.config['N_EPOCHS']*self.config['STEP_PERC'])

        
        



        # lr = CustomSchedule(self.d_model, 
        #      warmup_steps=1190*self.config['N_EPOCHS']*self.config['WARMUP_PERC'],
        #      decay_step=1190*self.config['N_EPOCHS']*self.config['STEP_PERC'])



        optimizer = tfa.optimizers.AdamW(learning_rate=lr, weight_decay=self.config['WEIGHT_DECAY'])

        self.model.compile(optimizer=optimizer,
                           loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.1),
                           metrics=[tf.keras.metrics.CategoricalAccuracy(name="accuracy")],
                           )

        self.name_model_bin = self.config['MODEL_NAME'] + '_' + self.config['MODEL_SIZE'] + '_' + str(self.split) + '_' + str(self.fold)
        
        if self.features=='skeleton':

            self.new_bin_path=join(self.bin_path,self.dataset,self.features,self.pose,self.evaluation,self.config['MODEL_SIZE'],'{:04d}/'.format(trial_num))
        elif self.features=='rgb':
            self.new_bin_path=join(self.bin_path,self.dataset,self.features,self.backbone,self.evaluation,self.config['MODEL_SIZE'],'{:04d}/'.format(trial_num))

        else:

            self.new_bin_path=join(self.bin_path,self.dataset,self.features,self.arch,self.evaluation,self.config['MODEL_SIZE'],'{:04d}/'.format(trial_num))


        

        


        self.checkpointer = tf.keras.callbacks.ModelCheckpoint(self.new_bin_path+self.name_model_bin,
                                                               monitor="val_accuracy",
                                                               save_best_only=True,
                                                               save_weights_only=True)

    def get_data_pose(self):

        X_train=np.load(join(self.dataset_dire_pose,'train','features.npy'))
        y_train=np.load(join(self.dataset_dire_pose,'train','labels.npy'))
        X_test=np.load(join(self.dataset_dire_pose,'test','features.npy'))
        y_test=np.load(join(self.dataset_dire_pose,'test','labels.npy'))
        

    	
        
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,
                                                          test_size=0.1,
                                                          random_state=101,
                                                          stratify=y_train)
        
        y_train=to_categorical(y_train,self.num_class)
        y_val=to_categorical(y_val,self.num_class)
        y_test=to_categorical(y_test,self.num_class)        
        ds_train = tf.data.Dataset.from_tensor_slices((X_train, y_train))
        # for _ in range(10):
        #     ds_train.concatenate(ds_train.map(aug, num_parallel_calls=tf.data.experimental.AUTOTUNE))
        # ds_train1 = ds_train1.map(random_noise, num_parallel_calls=tf.data.experimental.AUTOTUNE)
        #ds_train = ds_train.concatenate(ds_train1)
        ds_train = ds_train.cache()
        #ds_train = ds_train.map(random_flip, num_parallel_calls=tf.data.experimental.AUTOTUNE)
        # ds_train = ds_train.map(random_noise, num_parallel_calls=tf.data.experimental.AUTOTUNE)
        ds_train = ds_train.shuffle(X_train.shape[0])
        ds_train = ds_train.batch(self.config['BATCH_SIZE'])
        self.ds_train_pose = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

        ds_val = tf.data.Dataset.from_tensor_slices((X_val, y_val))
        
        # for _ in range(10):
        #     ds_val.concatenate(ds_val.map(aug, num_parallel_calls=tf.data.experimental.AUTOTUNE))
        
        ds_val = ds_val.cache()
        ds_val = ds_val.batch(self.config['BATCH_SIZE'])
        self.ds_val_pose = ds_val.prefetch(tf.data.experimental.AUTOTUNE)

        ds_test = tf.data.Dataset.from_tensor_slices((X_test, y_test))
        # ds_test = ds_test.map(lambda x,y : one_hot(x,y,self.config['CLASSES']), 
        #                          num_parallel_calls=tf.data.experimental.AUTOTUNE)
        
        
        ds_test = ds_test.cache()

        ds_test = ds_test.batch(self.config['BATCH_SIZE'])
        self.ds_test_pose = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

    def get_data_rgb(self):

        # X_train=np.load(join(self.dataset_dire_rgb,'train','features.npy'))
        # y_train=np.load(join(self.dataset_dire_rgb,'train','labels.npy'))
        # X_test=np.load(join(self.dataset_dire_rgb,'test','features.npy'))
        # y_test=np.load(join(self.dataset_dire_rgb,'test','labels.npy'))
        
        
        

        num_samples=len(glob(self.dataset_dire_rgb+'/**/*.npy'))
       
    	
        

        batch_size=self.config['BATCH_SIZE']


        

        output_signature = (tf.TensorSpec(shape = (None, None, None, 3), dtype = tf.float32),
                    tf.TensorSpec(shape = (3), dtype = tf.int16))
        

        ds=tf.data.Dataset.from_generator(FrameGenerator(rgb_path=self.dataset_dire_rgb,training=True),output_signature=output_signature)


        self.train_len_rgb=np.math.ceil(0.70*num_samples)

        val_len_rgb=np.math.ceil(0.10*num_samples)

        test_len_rgb=num_samples-self.train_len_rgb-val_len_rgb


        self.ds_train_rgb=ds.take(self.train_len_rgb)
        self.ds_train_rgb=self.ds_train_rgb.cache('../cr/train/')
        self.ds_train_rgb=self.ds_train_rgb.batch(batch_size)
        self.ds_train_rgb=self.ds_train_rgb.prefetch(tf.data.experimental.AUTOTUNE)


        self.ds_val_rgb=ds.skip(self.train_len_rgb).take(val_len_rgb)
        self.ds_val_rgb=self.ds_val_rgb.cache('../cr/val/')
        self.ds_val_rgb=self.ds_val_rgb.batch(batch_size)
        self.ds_val_rgb=self.ds_val_rgb.prefetch(tf.data.experimental.AUTOTUNE)



        self.ds_test_rgb=ds.skip(self.train_len_rgb+val_len_rgb).take(test_len_rgb)

        self.ds_test_rgb=self.ds_test_rgb.cache('../cr/test/')
        self.ds_test_rgb=self.ds_test_rgb.batch(batch_size)
        self.ds_test_rgb=self.ds_test_rgb.prefetch(tf.data.experimental.AUTOTUNE)





        

       


    

    def get_data_fusion(self):
        X_train_rgb=np.load(join(self.dataset_dire_rgb,'train','features.npy'))
        print('X_train_rgb.shape: ',X_train_rgb.shape)
        y_train_rgb=np.load(join(self.dataset_dire_rgb,'train','labels.npy'))
        
        X_test_rgb=np.load(join(self.dataset_dire_rgb,'test','features.npy'))
        y_test=np.load(join(self.dataset_dire_rgb,'test','labels.npy'))

        X_train_pose=np.load(join(self.dataset_dire_pose,'train','features.npy'))
        y_train_pose=np.load(join(self.dataset_dire_pose,'train','labels.npy'))

        print('X_train_pose.shape: ',X_train_pose.shape)
        
        X_test_pose=np.load(join(self.dataset_dire_pose,'test','features.npy'))
        
        X_train_pose, X_val_pose, y_train, y_val = train_test_split(X_train_pose, y_train_pose,
                                                          test_size=0.1,
                                                          random_state=101,
                                                          stratify=y_train_pose)

        X_train_rgb, X_val_rgb, _, _ = train_test_split(X_train_rgb, y_train_rgb,
                                                          test_size=0.1,
                                                          random_state=101,
                                                          stratify=y_train_rgb)

        y_train=to_categorical(y_train,self.num_class)
        y_val=to_categorical(y_val,self.num_class)
        y_test=to_categorical(y_test,self.num_class)   

     

        



        ds_train=tf.data.Dataset.from_tensor_slices((X_train_pose,X_train_rgb,y_train)).map(combine_inputs)
        ds_train = ds_train.cache()
        ds_train = ds_train.shuffle(X_train_pose.shape[0])
        ds_train = ds_train.batch(self.config['BATCH_SIZE'])
        self.ds_train_fusion = ds_train.prefetch(tf.data.experimental.AUTOTUNE)


        ds_val=tf.data.Dataset.from_tensor_slices((X_val_pose,X_val_rgb,y_val)).map(combine_inputs)
        ds_val = ds_val.cache()
        ds_val = ds_val.shuffle(X_val_pose.shape[0])
        ds_val = ds_val.batch(self.config['BATCH_SIZE'])
        self.ds_val_fusion = ds_train.prefetch(tf.data.experimental.AUTOTUNE)


        ds_test=tf.data.Dataset.from_tensor_slices((X_test_pose,X_test_rgb,y_test)).map(combine_inputs)
        ds_test = ds_test.cache()
        
        ds_test = ds_test.batch(self.config['BATCH_SIZE'])
        self.ds_test_fusion = ds_test.prefetch(tf.data.experimental.AUTOTUNE)



    






        



    


        
        

        

        
        
    def get_random_hp(self):
        self.config['RN_STD'] = self.trial.suggest_discrete_uniform("RN_STD", 0.0, 0.05, 0.01)
        self.config['WEIGHT_DECAY'] = self.trial.suggest_discrete_uniform("WD", 1e-5, 1e-3, 1e-3)    
        self.config['N_EPOCHS'] = int(self.trial.suggest_discrete_uniform("EPOCHS",17,21,1))
        
        self.config['WARMUP_PERC'] = self.trial.suggest_discrete_uniform("WARMUP_PERC", 0.2, 0.4, 0.1)
        
        self.logger.save_log('\nRN_STD: {:.2e}'.format(self.config['RN_STD']))
        self.logger.save_log('EPOCHS: {}'.format(self.config['N_EPOCHS']))
        self.logger.save_log('WARMUP_PERC: {:.2e}'.format(self.config['WARMUP_PERC']))
        self.logger.save_log('WEIGHT_DECAY: {:.2e}\n'.format(self.config['WEIGHT_DECAY']))
        
    def do_training(self,trial_num=0):


        if self.features=='skeleton':
            self.get_data_pose()
            self.get_model(trial_num)
            history = self.model.fit(self.ds_train_pose,
                epochs=self.config['N_EPOCHS'], initial_epoch=0,
                validation_data=self.ds_val_pose,
                callbacks=[self.checkpointer], verbose=self.config['VERBOSE'])

            self.best_bin_path=join(self.bin_path,self.dataset,self.features,self.pose,self.evaluation,self.config['MODEL_SIZE'],'{:04d}/'.format(trial_num))

            tf.keras.backend.clear_session()
            self.model.load_weights(self.best_bin_path+self.name_model_bin) 
            
                
            _, accuracy_test = self.model.evaluate(self.ds_test_pose)

            
            
            X, y_true = tuple(zip(*self.ds_test))
            y_pred = np.argmax(tf.nn.softmax(self.model.predict(tf.concat(X, axis=0)), axis=-1),axis=1)
            y_true=np.argmax(y_true,axis=0)
            
            
            y_true=np.load(join(self.dataset_dire_pose,'test','labels.npy'))
            y_pred=np.argmax(self.model.predict(self.ds_test_pose),axis=-1)
            
            class_report=classification_report(y_true,y_pred)


            balanced_accuracy = sklearn.metrics.balanced_accuracy_score(y_true,y_pred)

            text = f"{self.config['MODEL_SIZE']}:  Accuracy Test: {accuracy_test} <> Balanced Accuracy: {balanced_accuracy}\n"

            
            
            text1 = 'Classification Report \n{}'.format(class_report)
            
            self.logger.save_log(text)
            self.logger.save_log(text1)
            return accuracy_test, balanced_accuracy,history.history


            #print('len :',len(self.ds_train))
        elif self.features=='rgb':
            if self.backbone=='raw':
                self.get_data_rgb()

                
                
            else:
                self.get_data_rgb()

            

            self.get_model(trial_num)

            history = self.model.fit(self.ds_train_rgb,
                epochs=self.config['N_EPOCHS'], initial_epoch=0,
                validation_data=self.ds_val_rgb,
                callbacks=[self.checkpointer], verbose=self.config['VERBOSE'])

            self.best_bin_path=join(self.bin_path,self.dataset,self.features,self.backbone,self.evaluation,self.config['MODEL_SIZE'],'{:04d}/'.format(trial_num))


            tf.keras.backend.clear_session()
            self.model.load_weights(self.best_bin_path+self.name_model_bin) 
            _, accuracy_test = self.model.evaluate(self.ds_test_rgb)
            X, y_true = tuple(zip(*self.ds_test_rgb))
            y_true=tf.concat(y_true, axis=0)
            y_pred = np.argmax(tf.nn.softmax(self.model.predict(tf.concat(X, axis=0)), axis=-1),axis=1)
            y_true=np.argmax(y_true,axis=1)
            print('Shape: ',y_true.shape)
            
            
            
            
            class_report=classification_report(y_true,y_pred)


            balanced_accuracy = sklearn.metrics.balanced_accuracy_score(y_true,y_pred)

            text = f"{self.config['MODEL_SIZE']}:  Accuracy Test: {accuracy_test} <> Balanced Accuracy: {balanced_accuracy}\n"

            
            
            text1 = 'Classification Report \n{}'.format(class_report)
            
            self.logger.save_log(text)
            self.logger.save_log(text1)
            return accuracy_test, balanced_accuracy,history.history
                
            
                
        



            

        else:
            self.get_data_fusion()

            self.get_model(trial_num)

            history = self.model.fit(self.ds_train_fusion,
                epochs=self.config['N_EPOCHS'], initial_epoch=0,
                validation_data=self.ds_val_fusion,
                callbacks=[self.checkpointer], verbose=self.config['VERBOSE'])

            self.best_bin_path=join(self.bin_path,self.dataset,self.features,self.arch,self.evaluation,self.config['MODEL_SIZE'],'{:04d}/'.format(trial_num))


            tf.keras.backend.clear_session()
            self.model.load_weights(self.best_bin_path+self.name_model_bin) 
            
                
            _, accuracy_test = self.model.evaluate(self.ds_test_fusion)

            y_true=np.load(join(self.dataset_dire_rgb,'test','labels.npy'))
            y_pred=np.argmax(self.model.predict(self.ds_test_fusion),axis=-1)
            # #print('y.shape :', y_pred.shape)

            
            class_report=classification_report(y_true,y_pred)


            balanced_accuracy = sklearn.metrics.balanced_accuracy_score(y_true,y_pred)

            text = f"{self.config['MODEL_SIZE']}:  Accuracy Test: {accuracy_test} <> Balanced Accuracy: {balanced_accuracy}\n"
            
            text1 = 'Classification Report \n{}'.format(class_report)
            
            self.logger.save_log(text)
            self.logger.save_log(text1)
            return accuracy_test,balanced_accuracy,history.history


        
    
    def start_training(self, trial):
            
        
        acc,_,_ = self.do_training()
        



In [5]:
from datetime import datetime

now = datetime.now()
config = read_yaml('config.yaml')
logger = Logger(config['LOG_DIR']+now.strftime("%y%m%d%H%M%S"))

In [6]:
trainer = Trainer(config, logger)

: 